In [6]:
def calculate_bbox_area(bbox):
    """
    Calculate the area of a bounding box.
    bbox: (x1, y1, x2, y2) where (x1, y1) is top-left and (x2, y2) is bottom-right.
    """
    x1, y1, x2, y2, _ = bbox
    return (x2 - x1) * (y2 - y1)

def calculate_intersection(bbox1, bbox2):
    """
    Calculate the intersection area between two bounding boxes.
    bbox1: (x1, y1, x2, y2)
    bbox2: (x3, y3, x4, y4)
    """
    x1 = max(bbox1[0], bbox2[0])
    y1 = max(bbox1[1], bbox2[1])
    x2 = min(bbox1[2], bbox2[2])
    y2 = min(bbox1[3], bbox2[3])
    
    # Check if there is no intersection
    if x2 < x1 or y2 < y1:
        return 0.0
    
    return (x2 - x1) * (y2 - y1)

def calculate_center_distance(bbox1, bbox2):
    """
    Calculate the Euclidean distance between the centers of two bounding boxes.
    bbox1: (x1, y1, x2, y2)
    bbox2: (x3, y3, x4, y4)
    """
    center1_x = (bbox1[0] + bbox1[2]) / 2
    center1_y = (bbox1[1] + bbox1[3]) / 2
    center2_x = (bbox2[0] + bbox2[2]) / 2
    center2_y = (bbox2[1] + bbox2[3]) / 2
    
    return math.sqrt((center2_x - center1_x)**2 + (center2_y - center1_y)**2)

def calculate_diou(bbox1, bbox2):
    """
    Calculate the DIoU (Distance Intersection over Union) between two bounding boxes.
    bbox1: (x1, y1, x2, y2)
    bbox2: (x3, y3, x4, y4)
    """
    # Calculate intersection and union
    intersection = calculate_intersection(bbox1, bbox2)
    area1 = calculate_bbox_area(bbox1)
    area2 = calculate_bbox_area(bbox2)
    union = area1 + area2 - intersection
    
    # Calculate IoU
    iou = intersection / union if union != 0 else 0.0
    
    # Calculate the Euclidean distance between centers
    center_distance = calculate_center_distance(bbox1, bbox2)
    
    # Calculate the diagonal length of the smallest enclosing box
    x1 = min(bbox1[0], bbox2[0])
    y1 = min(bbox1[1], bbox2[1])
    x2 = max(bbox1[2], bbox2[2])
    y2 = max(bbox1[3], bbox2[3])
    diagonal_length = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
    # Calculate DIoU
    diou = iou - (center_distance**2) / (diagonal_length**2)
    
    return diou

In [171]:
import numpy as np

def diou(bbox1, bbox2):
    """
    Compute DIoU for a single pair of bounding boxes.
    :param bbox1: predict of bbox (4,) (x1, y1, x2, y2)
    :param bbox2: groundtruth of bbox (4,) (x1, y1, x2, y2)
    :return: DIoU value
    """
    # Calculate the intersection box
    xx1 = max(bbox1[0], bbox2[0])
    yy1 = max(bbox1[1], bbox2[1])
    xx2 = min(bbox1[2], bbox2[2])
    yy2 = min(bbox1[3], bbox2[3])
    
    w = max(0., xx2 - xx1)
    h = max(0., yy2 - yy1)
    wh = w * h
    
    # Calculate union area
    area1 = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
    area2 = (bbox2[2] - bbox2[0]) * (bbox2[3] - bbox2[1])
    union = area1 + area2 - wh
    
    # Calculate IoU
    iou = wh / union if union > 0 else 0
    
    # Calculate the center points of the bounding boxes
    centerx1 = (bbox1[0] + bbox1[2]) / 2.0
    centery1 = (bbox1[1] + bbox1[3]) / 2.0
    centerx2 = (bbox2[0] + bbox2[2]) / 2.0
    centery2 = (bbox2[1] + bbox2[3]) / 2.0
    
    # Calculate the distance between centers
    inner_diag = (centerx1 - centerx2) ** 2 + (centery1 - centery2) ** 2
    
    # Calculate the diagonal length of the smallest enclosing box
    xxc1 = min(bbox1[0], bbox2[0])
    yyc1 = min(bbox1[1], bbox2[1])
    xxc2 = max(bbox1[2], bbox2[2])
    yyc2 = max(bbox1[3], bbox2[3])
    
    outer_diag = (xxc2 - xxc1) ** 2 + (yyc2 - yyc1) ** 2
    
    # Calculate DIoU
    diou = iou - inner_diag / outer_diag if outer_diag > 0 else iou
    
    return (diou + 1) / 2.0  # Resize from (-1,1) to (0,1)

def bbsindex(bbox1, bbox2, iou_only=False):
    """
    Calculate the association cost based on IoU and box similarity for individual bounding boxes.
    :param bbox1: predict of bbox (4,) (x1, y1, x2, y2)
    :param bbox2: groundtruth of bbox (4,) (x1, y1, x2, y2)
    :param iou_only: if True, only compute IoU
    :return: cost value
    """
    eps = 1e-7

    # Calculate the intersection area
    xx1 = max(bbox1[0], bbox2[0])
    yy1 = max(bbox1[1], bbox2[1])
    xx2 = min(bbox1[2], bbox2[2])
    yy2 = min(bbox1[3], bbox2[3])
    
    w_intersection = max(0., xx2 - xx1)
    h_intersection = max(0., yy2 - yy1)
    intersection = w_intersection * h_intersection

    # Calculate the union area
    area1 = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
    area2 = (bbox2[2] - bbox2[0]) * (bbox2[3] - bbox2[1])
    union = area1 + area2 - intersection + eps

    # Calculate IoU
    iou = intersection / union

    if iou_only:
        return 1.0 - iou

    # Calculate the DIoU using the diou function
    diou_val = diou(bbox1, bbox2)

    # Calculate box similarity
    delta_w = abs((bbox2[2] - bbox2[0]) - (bbox1[2] - bbox1[0]))
    delta_h = abs((bbox2[3] - bbox2[1]) - (bbox1[3] - bbox1[1]))

    sw = w_intersection / (w_intersection + delta_w + eps)
    sh = h_intersection / (h_intersection + delta_h + eps)

    # Calculate the BBSI
    bbsi = diou_val + sh + sw

    # Normalize the BBSI
    cost = bbsi / 3.0

    return cost

def compute_iou(box1, box2):
    """
    Compute the Intersection over Union (IoU) between two bounding boxes.
    Parameters:
    box1, box2: list or tuple of length 4
        The coordinates of the bounding boxes in the format (x1, y1, x2, y2).
    Returns:
    float
        The IoU between the two bounding boxes.
    """
    # Unpack the coordinates
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2

    # Calculate the coordinates of the intersection rectangle
    x1_inter = max(x1_1, x1_2)
    y1_inter = max(y1_1, y1_2)
    x2_inter = min(x2_1, x2_2)
    y2_inter = min(y2_1, y2_2)

    # Compute the area of the intersection rectangle
    inter_width = max(0, x2_inter - x1_inter)
    inter_height = max(0, y2_inter - y1_inter)
    inter_area = inter_width * inter_height

    # Compute the area of both bounding boxes
    box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
    box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)

    # Compute the IoU
    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou

def compute_diou(box1, box2):
    """
    Compute the Distance-IoU (DIoU) between two bounding boxes.

    Parameters:
    box1, box2: list or tuple of length 4
        The coordinates of the bounding boxes in the format (x1, y1, x2, y2).

    Returns:
    float
        The DIoU between the two bounding boxes.
    """
    # Compute IoU
    iou = compute_iou(box1, box2)

    # Compute the center points of the boxes
    center_x1 = (box1[0] + box1[2]) / 2
    center_y1 = (box1[1] + box1[3]) / 2
    center_x2 = (box2[0] + box2[2]) / 2
    center_y2 = (box2[1] + box2[3]) / 2

    # Compute the Euclidean distance between the centers
    center_distance = math.sqrt((center_x1 - center_x2) ** 2 + (center_y1 - center_y2) ** 2)

    # Compute the diagonal distance of the smallest enclosing box
    x1_c = min(box1[0], box2[0])
    y1_c = min(box1[1], box2[1])
    x2_c = max(box1[2], box2[2])
    y2_c = max(box1[3], box2[3])
    diagonal_distance = math.sqrt((x2_c - x1_c) ** 2 + (y2_c - y1_c) ** 2)

    # Compute DIoU
    diou = iou - (center_distance ** 2) / (diagonal_distance ** 2)

    return diou

import math
def compute_eiou(box1, box2):
    """
    Compute the Efficient IoU (EIoU) between two bounding boxes.

    Parameters:
    box1, box2: list or tuple of length 4
        The coordinates of the bounding boxes in the format (x1, y1, x2, y2).

    Returns:
    float
        The EIoU between the two bounding boxes.
    """
    # Compute IoU
    iou = compute_iou(box1, box2)

    # Compute the center points of the boxes
    center_x1 = (box1[0] + box1[2]) / 2
    center_y1 = (box1[1] + box1[3]) / 2
    center_x2 = (box2[0] + box2[2]) / 2
    center_y2 = (box2[1] + box2[3]) / 2

    # Compute the Euclidean distance between the centers
    center_distance = math.sqrt((center_x1 - center_x2) ** 2 + (center_y1 - center_y2) ** 2)

    # Compute the diagonal distance of the smallest enclosing box
    x1_c = min(box1[0], box2[0])
    y1_c = min(box1[1], box2[1])
    x2_c = max(box1[2], box2[2])
    y2_c = max(box1[3], box2[3])
    diagonal_distance = math.sqrt((x2_c - x1_c) ** 2 + (y2_c - y1_c) ** 2)

    # Compute the width and height differences
    w1 = box1[2] - box1[0]
    h1 = box1[3] - box1[1]
    w2 = box2[2] - box2[0]
    h2 = box2[3] - box2[1]

    width_diff = (w1 - w2) ** 2
    height_diff = (h1 - h2) ** 2

    # Compute the penalty terms for width and height
    cw = x2_c - x1_c  # Width of the smallest enclosing box
    ch = y2_c - y1_c  # Height of the smallest enclosing box

    # Compute EIoU
    eiou = iou - (center_distance ** 2) / (diagonal_distance ** 2) - (width_diff) / (cw ** 2) - (height_diff) / (ch ** 2)

    return eiou

In [181]:
import numpy as np
import math

def calculate_bbox_center(bbox):
    """
    Calculate the center of a bounding box.
    bbox: (x1, y1, x2, y2) where (x1, y1) is top-left and (x2, y2) is bottom-right.
    """
    x1, y1, x2, y2, _ = bbox
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    return (center_x, center_y)

def euclidean_distance(bbox1, bbox2):
    """
    Calculate the Euclidean distance between two centers.
    center1: (x1, y1)
    center2: (x2, y2)
    """
    center1 = calculate_bbox_center(bbox1)
    center2 = calculate_bbox_center(bbox2)
    return math.sqrt((center2[0] - center1[0])**2 + (center2[1] - center1[1])**2)

import numpy as np

def compute_ciou(box1, box2):
    """
    Compute the Complete Intersection over Union (CIoU) between two bounding boxes,
    with the height ratio multiplied into the final CIoU.

    Parameters:
    box1, box2: list or numpy array of shape (4,)
        The coordinates of the bounding boxes in the format [x1, y1, x2, y2],
        where (x1, y1) is the top-left corner and (x2, y2) is the bottom-right corner.

    Returns:
    ciou: float
        The CIoU between the two bounding boxes, scaled by the height ratio.
    """
    # Convert boxes to numpy arrays if they are not already
    box1 = np.array(box1)
    box2 = np.array(box2)

    # Calculate the coordinates of the intersection rectangle
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    # Calculate the area of the intersection rectangle
    inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)

    # Calculate the area of both bounding boxes
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Calculate the area of the union
    union_area = box1_area + box2_area - inter_area

    # Calculate IoU
    iou = inter_area / union_area if union_area != 0 else 0

    # Calculate the center points of the bounding boxes
    box1_center = np.array([(box1[0] + box1[2]) / 2, (box1[1] + box1[3]) / 2])
    box2_center = np.array([(box2[0] + box2[2]) / 2, (box2[1] + box2[3]) / 2])

    # Calculate the Euclidean distance between the centers
    center_distance = np.linalg.norm(box1_center - box2_center)

    # Calculate the diagonal distance of the smallest enclosing box
    enclose_x1 = min(box1[0], box2[0])
    enclose_y1 = min(box1[1], box2[1])
    enclose_x2 = max(box1[2], box2[2])
    enclose_y2 = max(box1[3], box2[3])
    enclose_diagonal = np.sqrt((enclose_x2 - enclose_x1)**2 + (enclose_y2 - enclose_y1)**2)

    # Calculate the aspect ratio term
    v = (4 / (np.pi ** 2)) * (np.arctan((box1[2] - box1[0]) / (box1[3] - box1[1])) - np.arctan((box2[2] - box2[0]) / (box2[3] - box2[1]))) ** 2

    # Calculate the alpha parameter
    alpha = v / (1 - iou + v + 1e-6)

    # Calculate CIoU
    ciou = iou - (center_distance ** 2) / (enclose_diagonal ** 2) - alpha * v

    # Calculate the height ratio
    #height_ratio = min((box1[3] - box1[1]) / (box2[3] - box2[1]), (box2[3] - box2[1]) / (box1[3] - box1[1]))

    # Multiply the height ratio into the CIoU
    #ciou *= height_ratio

    return ciou

def calculate_cost_matrix_ciou(bounding_boxes, tracks, alpha=1):
    
    # Initialize cost matrix
    num_boxes = len(bounding_boxes)
    num_tracks = len(tracks)
    cost_matrix = np.zeros((num_boxes, num_tracks))

    # Calculate cost matrix
    for i, box in enumerate(bounding_boxes):
        for j, track in enumerate(tracks):
            giou_ar = bbsindex(box, track) + compute_ciou(box, track) #compute_ciou(box, track) #+ calculate_diou(box, track) #euclidean_distance(box, track)
            cost_matrix[i, j] = giou_ar  # Negative for cost minimization

    return cost_matrix

In [182]:
import numpy as np

def calculate_cost_matrix(bounding_boxes, tracks, alpha=1):
    """
    Calculate the cost matrix using the negative of GIoU with aspect ratio adjustment.

    Parameters:
        bounding_boxes (list): List of bounding boxes, each in [x_min, y_min, x_max, y_max].
        tracks (list): List of tracks, each in [x_min, y_min, x_max, y_max].
        alpha (float): Weighting factor for GIoU and aspect ratio similarity.

    Returns:
        np.ndarray: Cost matrix of shape (len(bounding_boxes), len(tracks)).
    """
    def calculate_giou_ar(boxA, boxB):
        # Calculate intersection coordinates
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])

        # Intersection area
        inter_width = max(0, xB - xA)
        inter_height = max(0, yB - yA)
        inter_area = inter_width * inter_height

        # Areas of boxA and boxB
        areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
        areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

        # Union area
        union_area = areaA + areaB - inter_area

        # IoU
        iou = inter_area / union_area if union_area > 0 else 0

        # Smallest enclosing box
        xC_min = min(boxA[0], boxB[0])
        yC_min = min(boxA[1], boxB[1])
        xC_max = max(boxA[2], boxB[2])
        yC_max = max(boxA[3], boxB[3])

        enclosing_width = xC_max - xC_min
        enclosing_height = yC_max - yC_min
        enclosing_area = enclosing_width * enclosing_height

        # GIoU
        giou = iou - ((enclosing_area - union_area) / enclosing_area if enclosing_area > 0 else 0)

        # Aspect ratio similarity
        rA = (boxA[2] - boxA[0]) / (boxA[3] - boxA[1] + 1e-6)
        rB = (boxB[2] - boxB[0]) / (boxB[3] - boxB[1] + 1e-6)
        aspect_ratio_similarity = 1 - abs(rA - rB) / max(rA, rB, 1e-6)

        # GIoU with aspect ratio adjustment
        giou_ar = alpha * giou + (1 - alpha) * aspect_ratio_similarity

        return giou_ar

    # Initialize cost matrix
    num_boxes = len(bounding_boxes)
    num_tracks = len(tracks)
    cost_matrix = np.zeros((num_boxes, num_tracks))

    # Calculate cost matrix
    for i, box in enumerate(bounding_boxes):
        for j, track in enumerate(tracks):
            giou_ar = calculate_giou_ar(box, track)
            cost_matrix[i, j] = giou_ar  # Negative for cost minimization

    return cost_matrix

In [183]:
import pygmtools as pygm
import numpy as np

# Input data
bounding_boxes = np.array([
    
    [163.61, 624.68, 481.58, 842.4, 0.81364],
    [499.51, 618.71, 734.06, 782.65, 0.80594],
    [15.217, 596.98, 255.37, 746.1, 0.78787],
    [1145.3, 121.67, 1342.1, 262.05, 0.77928],
    [745.83, 598.71, 976.46, 807.44, 0.77625],
    [1500.5, 613.43, 1760.8, 791.44, 0.77464],
    [1159.6, 676.4, 1384.3, 822.95, 0.7472],
    [1288.7, 557, 1519, 750.16, 0.69267],
    [1669.9, 177.3, 1888.9, 391.73, 0.74538],
    [1421.1, 7.7202, 1685.3, 220.28, 0.8169],
    [1145.3, 121.67, 1342.1, 262.05, 0.77928],
    [61.481, 0, 327.97, 90.72, 0.67228]
])

# Tracks
tracks = np.array([
    [1174.6, 965.56, 1263.6, 1099.9, 0],
    [1581.7, 996.99, 1811.7, 1221.1, 0],
    [1421.1, 1063.7, 1522.8, 1117.6, 0],
    [892.25, 895.87, 1119, 1104.2, 0],
    [369.47, 776.34, 452.08, 1013.5, 0],
    [1196.1, 537.93, 1432.9, 713.52, 0],
    [1559.1, 470.24, 1784.9, 697.03, 0],
    [1320.5, 392.62, 1585.3, 655.64, 0],
    [1749.7, 23.409, 1948.5, 266.07, 0],
    [773.32, 436.37, 1020.4, 675.4, 0],
    [195.53, 437.87, 520.25, 704.14, 0],
    [773.32, 436.37, 1020.4, 675.4, 0],
    [549.21, 442.45, 777.73, 647.1, 0],
    [47.793, 407.75, 299.87, 581.25, 0],
    [32.801, 350, 91.73, 533.87, 0],
    [1178.3, -12.17, 1421.1, 157.05, 0],
    [53.311, 400.54, 303.12, 572.61, 0],
])

In [184]:

def compute_iou(box1, box2):
    """
    Compute the Intersection over Union (IoU) between two bounding boxes.
    Parameters:
    box1, box2: list or tuple of length 4
        The coordinates of the bounding boxes in the format (x1, y1, x2, y2).
    Returns:
    float
        The IoU between the two bounding boxes.
    """
    # Unpack the coordinates
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2

    # Calculate the coordinates of the intersection rectangle
    x1_inter = max(x1_1, x1_2)
    y1_inter = max(y1_1, y1_2)
    x2_inter = min(x2_1, x2_2)
    y2_inter = min(y2_1, y2_2)

    # Compute the area of the intersection rectangle
    inter_width = max(0, x2_inter - x1_inter)
    inter_height = max(0, y2_inter - y1_inter)
    inter_area = inter_width * inter_height

    # Compute the area of both bounding boxes
    box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
    box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)

    # Compute the IoU
    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou

def compute_eiou(box1, box2):
    """
    Compute the Efficient IoU (EIoU) between two bounding boxes.

    Parameters:
    box1, box2: list or tuple of length 4
        The coordinates of the bounding boxes in the format (x1, y1, x2, y2).

    Returns:
    float
        The EIoU between the two bounding boxes.
    """
    # Compute IoU
    iou = compute_iou(box1, box2)

    # Compute the center points of the boxes
    center_x1 = (box1[0] + box1[2]) / 2
    center_y1 = (box1[1] + box1[3]) / 2
    center_x2 = (box2[0] + box2[2]) / 2
    center_y2 = (box2[1] + box2[3]) / 2

    # Compute the Euclidean distance between the centers
    center_distance = math.sqrt((center_x1 - center_x2) ** 2 + (center_y1 - center_y2) ** 2)

    # Compute the diagonal distance of the smallest enclosing box
    x1_c = min(box1[0], box2[0])
    y1_c = min(box1[1], box2[1])
    x2_c = max(box1[2], box2[2])
    y2_c = max(box1[3], box2[3])
    diagonal_distance = math.sqrt((x2_c - x1_c) ** 2 + (y2_c - y1_c) ** 2)

    # Compute the width and height differences
    w1 = box1[2] - box1[0]
    h1 = box1[3] - box1[1]
    w2 = box2[2] - box2[0]
    h2 = box2[3] - box2[1]

    width_diff = (w1 - w2) ** 2
    height_diff = (h1 - h2) ** 2

    # Compute the penalty terms for width and height
    cw = x2_c - x1_c  # Width of the smallest enclosing box
    ch = y2_c - y1_c  # Height of the smallest enclosing box

    # Compute EIoU
    eiou = iou - (center_distance ** 2) / (diagonal_distance ** 2) - (width_diff) / (cw ** 2) - (height_diff) / (ch ** 2)

    return eiou

In [185]:
n_bboxes1 = bounding_boxes.shape[0]
n_bboxes2 = tracks.shape[0]

Aff_dist_1 = calculate_cost_matrix_ciou(bounding_boxes,bounding_boxes) #calculate_cost_matrix(bounding_boxes, bounding_boxes)
Aff_dist_2 = calculate_cost_matrix_ciou(tracks, tracks) #calculate_cost_matrix(tracks, tracks)

print("Aff_dist_1: ", np.round(Aff_dist_1, decimals=3))
print("Aff_dist_2: ", np.round(Aff_dist_2, decimals=3))
print()
conn1, edge1 = pygm.utils.dense_to_sparse(Aff_dist_1)
conn2, edge2 = pygm.utils.dense_to_sparse(Aff_dist_2)

import functools
gaussian_aff = functools.partial(pygm.utils.gaussian_aff_fn, sigma=0.09)  # set affinity function
graph = pygm.utils.build_aff_mat(None, edge1, conn1, None, edge2, conn2, [n_bboxes1], None, [n_bboxes2], None, edge_aff_fn=gaussian_aff)
print("graph: ", np.round(graph, decimals=3))
matching = pygm.classic_solvers.rrwm(graph, [n_bboxes1], [n_bboxes2])
print("matching: ", matching)


Aff_dist_1:  [[ 2.     0.143  0.535 -0.531  0.013 -0.331 -0.293 -0.269 -0.626 -0.569
  -0.531 -0.225]
 [ 0.143  2.    -0.027 -0.497  0.171 -0.265 -0.158 -0.226 -0.608 -0.543
  -0.497 -0.5  ]
 [ 0.535 -0.027  2.    -0.625 -0.232 -0.406 -0.298 -0.393 -0.702 -0.646
  -0.625 -0.246]
 [-0.531 -0.497 -0.625  2.    -0.409 -0.412 -0.207 -0.15  -0.206  0.026
   2.    -0.6  ]
 [ 0.013  0.171 -0.232 -0.409  2.    -0.193 -0.043 -0.056 -0.531 -0.466
  -0.409 -0.524]
 [-0.331 -0.265 -0.406 -0.412 -0.193  2.     0.052  0.401 -0.039 -0.07
  -0.412 -0.662]
 [-0.293 -0.158 -0.298 -0.207 -0.043  0.052  2.     0.647 -0.416 -0.432
  -0.207 -0.63 ]
 [-0.269 -0.226 -0.393 -0.15  -0.056  0.401  0.647  2.    -0.303 -0.102
  -0.15  -0.632]
 [-0.626 -0.608 -0.702 -0.206 -0.531 -0.039 -0.416 -0.303  2.     0.323
  -0.206 -0.697]
 [-0.569 -0.543 -0.646  0.026 -0.466 -0.07  -0.432 -0.102  0.323  2.
   0.026 -0.504]
 [-0.531 -0.497 -0.625  2.    -0.409 -0.412 -0.207 -0.15  -0.206  0.026
   2.    -0.6  ]
 [-0.225 -0.

In [186]:
matching = pygm.hungarian(matching)#, [n_bboxes1], [n_bboxes2])

print("Matching: ", np.round(matching, decimals=3))

Matching:  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [187]:
# Find matched bounding boxes and tracks
matched_bbox_indices = np.where(matching == 1)[0]
matched_track_indices = np.where(matching == 1)[1]

# Find non-matched bounding boxes
non_matched_bbox_indices = set(range(len(bounding_boxes))) - set(matched_bbox_indices)
non_matched_bboxes = bounding_boxes[list(non_matched_bbox_indices)]

# Find non-matched tracks
non_matched_track_indices = set(range(len(tracks))) - set(matched_track_indices)
non_matched_tracks = tracks[list(non_matched_track_indices)]

print("Non-matched bounding boxes:")
print(non_matched_bboxes)

print("\nNon-matched tracks:")
print(non_matched_tracks)

Non-matched bounding boxes:
[]

Non-matched tracks:
[[ 892.25   895.87  1119.    1104.2      0.   ]
 [1749.7     23.409 1948.5    266.07     0.   ]
 [ 773.32   436.37  1020.4    675.4      0.   ]
 [  47.793  407.75   299.87   581.25     0.   ]
 [  32.801  350.      91.73   533.87     0.   ]]


In [188]:
matched_bbox_indices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int64)

In [189]:
matched_track_indices

array([10, 12, 16, 15,  9,  6,  5,  7,  2,  1,  0,  4], dtype=int64)

In [190]:
matches = [[bbox_idx, track_idx] for bbox_idx, track_idx in zip(matched_bbox_indices, matched_track_indices)]

In [191]:
# Find matched bounding boxes and tracks
matched_bbox_indices = np.where(matching == 1)[0]
matched_track_indices = np.where(matching == 1)[1]

# Print the matches
print("Matches:")
for bbox_idx, track_idx in zip(matched_bbox_indices, matched_track_indices):
    bbox = bounding_boxes[bbox_idx]
    track = tracks[track_idx]
    print(f"Box: {bbox}  <-->  Track: {track}: {bbsindex(bbox[0:4], track[0:4])} CIoU: {compute_ciou(bbox[0:4], track[0:4])}")

Matches:
Box: [1.6361e+02 6.2468e+02 4.8158e+02 8.4240e+02 8.1364e-01]  <-->  Track: [195.53 437.87 520.25 704.14   0.  ]: 0.7118649854971055 CIoU: 0.07581995052203247
Box: [499.51    618.71    734.06    782.65      0.80594]  <-->  Track: [549.21 442.45 777.73 647.1    0.  ]: 0.6145029870460111 CIoU: -0.0715453066933846
Box: [ 15.217   596.98    255.37    746.1       0.78787]  <-->  Track: [ 53.311 400.54  303.12  572.61    0.   ]: 0.4550934517151953 CIoU: -0.1782231580919457
Box: [1.1453e+03 1.2167e+02 1.3421e+03 2.6205e+02 7.7928e-01]  <-->  Track: [1178.3   -12.17 1421.1   157.05    0.  ]: 0.6067365240857148 CIoU: -0.022905420027136702
Box: [7.4583e+02 5.9871e+02 9.7646e+02 8.0744e+02 7.7625e-01]  <-->  Track: [ 773.32  436.37 1020.4   675.4     0.  ]: 0.7243572966047739 CIoU: 0.06237628895315719
Box: [1.5005e+03 6.1343e+02 1.7608e+03 7.9144e+02 7.7464e-01]  <-->  Track: [1559.1   470.24 1784.9   697.03    0.  ]: 0.682321014790332 CIoU: 0.12276718097243432
Box: [1.1596e+03 6.7640e+0

In [192]:
#compute_eiou([ 1.1453e+03, 1.2167e+02, 1.3421e+03, 2.6205e+02], [ 1174.6,   965.56, 1263.6,  1099.9  ])

In [196]:
def graph_matching(detections, tracks):
    """
    Performs graph matching between bounding boxes and tracks.

    Args:
        detections (np.ndarray): Array of bounding boxes.
        tracks (np.ndarray): Array of tracks.

    Returns:
        tuple: Contains matched pairs and lists of missing bounding boxes and tracks.
    """
    if(len(tracks)==0):
        return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)
    if(len(detections)==0):
        return np.empty((0,2),dtype=int), np.empty((0,5),dtype=int), np.arange(len(tracks))
    
    n_bboxes = detections.shape[0]
    n_tracks = tracks.shape[0]

    # Calculate affinity distance matrices
    Aff_dist_bboxes = calculate_cost_matrix_ciou(detections, detections)
    Aff_dist_tracks = calculate_cost_matrix_ciou(tracks, tracks)

    print(Aff_dist_bboxes)
    print(Aff_dist_tracks)

    # Convert dense matrices to sparse format
    conn1, edge1 = pygm.utils.dense_to_sparse(Aff_dist_bboxes)
    conn2, edge2 = pygm.utils.dense_to_sparse(Aff_dist_tracks)

    # Set affinity function and build affinity matrix
    gaussian_aff = functools.partial(pygm.utils.gaussian_aff_fn, sigma=0.009) #0.09
    graph = pygm.utils.build_aff_mat(None, edge1, conn1, None, edge2, conn2, [n_bboxes], None, [n_tracks], None, edge_aff_fn=gaussian_aff)

    # Perform graph matching using RRWM and Hungarian algorithms
    matching = pygm.classic_solvers.rrwm(graph, [n_bboxes], [n_tracks])
    matching = pygm.hungarian(matching)
    print("Matching: ", matching)

    # Find matched bounding boxes and tracks
    matched_bbox_indices = np.where(matching == 1)[0]
    matched_track_indices = np.where(matching == 1)[1]
    
    #matches = [[bbox_idx, track_idx] for bbox_idx, track_idx in zip(matched_bbox_indices, matched_track_indices)]

    # Find missing bounding boxes and tracks
    missing_bbox_indices = list(set(range(n_bboxes)) - set(matched_bbox_indices))
    missing_track_indices = list(set(range(n_tracks)) - set(matched_track_indices))
    
    matches = []
    for bbox_idx, track_idx in zip(matched_bbox_indices, matched_track_indices):
        bbox = detections[bbox_idx]
        track = tracks[track_idx]
        print("bbsindex", bbsindex(bbox, track))
        print("compute_iou", compute_iou(bbox, track))

        if bbsindex(bbox, track) >= 0.60:
            matches.append([bbox_idx, track_idx])
        elif compute_iou(bbox, track) >= 0.50:
            matches.append([bbox_idx, track_idx])
        else:
            missing_bbox_indices.append(bbox_idx)
            missing_track_indices.append(track_idx)


    return np.array(matches), np.array(missing_bbox_indices), np.array(missing_track_indices)

# Example usage
#matches, missing_bbox_indices, missing_track_indices = graph_matching(bounding_boxes, tracks)

## Test cases

In [205]:
# =========================== Case 1 ===========================
#tracks = np.array([[     1089.3,      16.738,      1396.9,      268.43     ],
# [     775.32,      17.578,      1109.3,      219.77     ],
# [     1484.8,      34.892,      1887.2,      402.99     ]])

#bounding_boxes = np.array([[     790.06,           0,      1100.4,      218.78],
# [       1101,     0.56648,     1389.4,      263.64     ],
# [     412.33,           0,      665.58,        80.3     ],
# [     1632.3,      15.669,      1850.3,      285.95     ]])

# =========================== Case 2 =========================== 
# Failed because of different bbounding box size
"""
tracks = np.array([
    [     1092.1,      90.973,      1413.6,      392.27    ],
    [     771.18,      48.057,      1122.4,      349.99    ],
    [       1525,      96.208,        1907,      537.59    ],
    [      391.2,      68.531,      692.68,      192.63    ]
])

bounding_boxes = np.array([
    [     781.62,      62.562,      1116.1,      362.37     ],
    [     1099.6,      126.26,      1407.4,         399     ],
    [     375.86,           0,      687.34,      210.32     ],
    [          0,           0,      199.38,       137.4     ],
    [       1649,      148.49,      1861.4,      377.07     ],
    [    0.39315,           0,      260.73,      122.44     ],
#    [     140.25,           0,      279.25,      52.679     ]
])"""

# =========================== Case 3 =========================== 
"""
tracks = np.array([[      794.6,      115.95,      1107.8,      414.87,     ],
 [     1114.7,      172.09,      1390.1,      445.84,     ],
 [     371.61,           0,      679.01,      262.44,     ],
 [          0,           0,      266.14,      186.11,     ],
 [     1650.4,      178.46,      1874.8,      513.89,     ]])

bounding_boxes = np.array([[     1100.1,      145.83,      1410.6,      448.46     ],
 [     777.68,      78.316,      1124.3,      415.38     ],
 [     1640.7,      185.32,      1858.3,      431.62     ],
 [     367.69,      8.8086,      701.59,      271.56     ],
 [     3.5089,      33.657,      202.49,      171.19     ]])"""

# =========================== Case 4 =========================== frame 26 Good
"""
tracks = np.array([
    [     807.87,      686.94,      1157.9,      1043.3     ],
    [    0.38459,      1.6927,      255.15,      278.04     ],
    [     1119.1,      735.76,      1429.9,      1060.9     ],
    [     725.44,           0,       988.9,      184.67     ],
    [    0.30798,      532.72,      315.99,      799.15     ],
    [     1324.4,           0,      1512.1,      180.35     ],
    [     1594.6,      745.09,      1892.8,      1085.8     ],
    [     1747.8,      138.76,      1919.5,      394.18     ],
    [      216.9,      73.537,      471.83,      341.89     ],
    [     981.13,      148.88,      1193.5,       279.1     ]])

bounding_boxes = np.array([
    [     1122.3,      733.32,      1430.2,      1050.1,     ],
    [     810.98,      684.58,      1158.1,      1039.2,     ],
    [     1619.1,      744.09,      1896.7,      1095.7,     ],
    [     387.26,      597.46,      737.16,      880.93,     ],
    [    -2.8963,      530.55,      321.67,      805.36,     ],
    [     1751.8,      120.89,      1922.3,      425.23,     ],
    [    -4.2382,      24.204,      254.18,      298.19,     ],
    [     791.63,      151.53,       951.1,      205.92,     ],
    [     747.67,      49.125,      977.97,      184.71,     ],
    [     1331.4,      48.585,      1512.6,      182.75,     ],
    [     210.16,      69.236,      461.39,      320.75,     ]
])"""

# =========================== Case 5 =========================== frame 26 Good
"""
bounding_boxes = np.array([
    [     1421.1,      7.7202,      1685.3,      220.28     ],
    [     163.61,      624.68,      481.58,       842.4     ],
    [     499.51,      618.71,      734.06,      782.65     ],
    [     15.217,      596.98,      255.37,       746.1     ],
    [     1145.3,      121.67,      1342.1,      262.05     ],
    [     745.83,      598.71,      976.46,      807.44     ],
    [     1500.5,      613.43,      1760.8,      791.44     ],
    [     1159.6,       676.4,      1384.3,      822.95     ],
    [     1669.9,       177.3,      1888.9,      391.73     ],
    [     1288.7,         557,        1519,      750.16     ],
    [     61.481,           0,      327.97,       90.72     ],
    #[   0.062759,      546.21,      60.611,      681.81     ],
    #[     801.07,           0,      1122.3,      108.66     ],
    #[     799.48,           0,      1030.2,      105.42     ],
    #[   0.036472,      520.48,      46.677,      683.29     ],
    #[     314.28,       927.1,      440.48,      1059.8     ],
    #[     326.62,      952.68,      437.64,        1069     ],
    #[      705.6,      966.81,      880.13,      1081.1     ]
])

tracks = np.array([
    [     1548.5,      947.03,      1788.2,      1146.6    ],
    [     1398.6,      1022.9,      1501.4,      1067.2    ],
    [     1139.3,      531.03,      1382.6,      682.43    ],
    [     1485.6,      436.78,      1722.4,      638.52    ],
    [     163.42,      521.73,       496.4,      737.04    ],
    [     723.96,      472.88,      977.77,      671.32    ],
    [     505.02,      489.67,      742.67,      669.81    ],
    [     19.756,      497.81,      271.11,      642.02    ],
    [     875.45,      910.96,      1107.2,        1075    ],
    [     1634.8,     -11.131,      1846.9,      207.91    ],
    [     1096.4,      17.823,      1309.1,      124.72    ]
])"""

In [220]:

bounding_boxes = np.array( [[     1421.1,      7.7202,      1685.3,      220.28     ],
 [     163.61,      624.68,      481.58,       842.4     ],
 [     499.51,      618.71,      734.06,      782.65     ],
 [     15.217,      596.98,      255.37,       746.1     ],
 [     1145.3,      121.67,      1342.1,      262.05     ],
 [     745.83,      598.71,      976.46,      807.44     ],
 [     1500.5,      613.43,      1760.8,      791.44     ],
 [     1159.6,       676.4,      1384.3,      822.95     ],
 [     1669.9,       177.3,      1888.9,      391.73     ],
 [     1288.7,         557,        1519,      750.16     ],
 [     61.481,           0,      327.97,       90.72     ]
])

tracks = np.array([[     1548.5,      947.03,      1788.2,      1146.6     ],
 [     1398.6,      1022.9,      1501.4,      1067.2     ],
 [      873.3,      901.86,      1109.3,      1085.7     ],
 [     356.92,      827.22,       455.7,      1050.3     ],
 [     1139.3,      531.03,      1382.6,      682.43     ],
 [     1485.6,      436.78,      1722.4,      638.52     ],
 [     162.04,      513.25,       497.7,      745.37     ],
 [      721.7,      466.08,      979.99,      678.03     ],
 [     505.02,      489.67,      742.67,      669.81     ],
 [     19.756,      497.81,      271.11,      642.02     ],
 [     1248.4,      380.71,      1525.7,      614.53     ],
 [    -2.4995,      440.76,      69.185,      614.22     ],
 [     1634.8,     -11.131,      1846.9,      207.91     ],
 [     1137.1,       91.61,      1287.7,      133.85     ],
 [       1146,      969.61,      1259.3,      1047.5     ]])

In [221]:
graph_matching(bounding_boxes, tracks)

[[ 2.         -0.56853779 -0.54327762 -0.64557215  0.02560336 -0.46638977
  -0.07015719 -0.43243414  0.32276096 -0.10188212 -0.50442753]
 [-0.56853779  2.          0.14304847  0.53494245 -0.5314243   0.01291972
  -0.33128752 -0.29306061 -0.62647417 -0.26940652 -0.22543888]
 [-0.54327762  0.14304847  2.         -0.027227   -0.49651679  0.17139197
  -0.2654634  -0.15760628 -0.6079962  -0.22587852 -0.49997758]
 [-0.64557215  0.53494245 -0.027227    2.         -0.62466029 -0.23216398
  -0.4056337  -0.29839266 -0.70177023 -0.39297934 -0.24554468]
 [ 0.02560336 -0.5314243  -0.49651679 -0.62466029  2.         -0.40932159
  -0.41201527 -0.20723451 -0.20647218 -0.14966642 -0.60044922]
 [-0.46638977  0.01291972  0.17139197 -0.23216398 -0.40932159  2.
  -0.19250349 -0.04285629 -0.53071334 -0.05562944 -0.5244269 ]
 [-0.07015719 -0.33128752 -0.2654634  -0.4056337  -0.41201527 -0.19250349
   2.          0.05166454 -0.03908137  0.40129255 -0.662085  ]
 [-0.43243414 -0.29306061 -0.15760628 -0.29839266

(array([[ 1,  6],
        [ 2,  8],
        [ 3,  9],
        [ 5,  7],
        [ 6,  5],
        [ 7,  4],
        [ 9, 10]], dtype=int64),
 array([ 0,  4,  8, 10], dtype=int64),
 array([ 2, 11, 12, 13,  0, 14,  1,  3], dtype=int64))